In [1]:
import MetaTrader5 as mt5 
import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import adfuller 
from datetime import datetime
import requests
import pandas as pd
import numpy as np
import warnings
import json
warnings.filterwarnings("ignore")
mt5.initialize()
# Replace following with your MT5 Account Login
account=51434456 # 
password="9UpBvVzc"
server = 'ICMarkets-Demo'

def get_rates(pair1, timeframe, x):
    pair1 = pd.DataFrame(mt5.copy_rates_from_pos(pair1, timeframe, 0, x))
    pair1['time'] = pd.to_datetime(pair1['time'], unit = 's')
    return pair1[['time','open', 'high', 'low', 'close']].set_index('time')

In [2]:
import requests
import xml.etree.ElementTree as ET
import pandas as pd

combined_dfs = {}

# Define the base URL for the BIS Stats API
base_url = 'https://stats.bis.org/api/v1'

count_codes = [['AUDUSD.a','AU'], ['USDCAD.a', 'CA'],
               ['USDCHF.a','CH'], ['GBPUSD.a', 'GB'],
               ['USDJPY.a', 'JP'], ['NZDUSD.a', 'NZ'], 
               ['EURUSD.a', 'XM']]

for country in count_codes:

    # Define the endpoint parameters
    flow = 'BIS,WS_EER_D,1.0'  # Example: Version 1.0 of the WS_EER_M domain, maintained by the BIS
    key = f'D.N.N.{country[1]}'
    start_period = '2000'  # Example: Start year 2000
    end_period = '2023'  # Example: End year 2020
    detail = 'full'  # Example: All data and documentation

    # Construct the endpoint URL
    endpoint_url = f'{base_url}/data/{flow}/{key}/all'

    # Define the query parameters
    query_params = {
        'startPeriod': start_period,
        'endPeriod': end_period,
        'detail': detail
    }

    # Make the GET request
    response = requests.get(endpoint_url, params=query_params)

    # Check for a successful response
    if response.status_code == 200:
        # Assign the text of the response to xml_data
        xml_data = response.text

        # Parse the XML data
        root = ET.fromstring(xml_data)

        # Initialize empty lists to store the data
        time_periods = []
        obs_values = []

        # Iterate through the XML and extract the desired information
        for obs in root.findall(".//Obs"):
            time_period = obs.get('TIME_PERIOD')
            obs_value = obs.get('OBS_VALUE')
            time_periods.append(time_period)
            obs_values.append(obs_value)

        # Create a DataFrame
        df = pd.DataFrame({
            'Time_Period': time_periods,
            'OBS_Value': obs_values
        })

        df['OBS_Value'] = df['OBS_Value'].replace('NaN', np.nan)
        # Drop rows with NaN values
        df.dropna(subset=['OBS_Value'], inplace=True)
        df['OBS_Value'] = df['OBS_Value'].astype(float)
        df['Time_Period'] = pd.to_datetime(df['Time_Period'])
        
        df = df.set_index('Time_Period')
        print(f"Getting {country[0]}'s rates")
        rates = get_rates(country[0], mt5.TIMEFRAME_D1, 2500)
        
        combined = pd.concat([df[-len(rates):], rates['close']], join = 'outer', axis = 1)
        
        combined_dfs[country[1]] = combined.dropna()

    else:
        print(f'Failed to retrieve data: {response.status_code}')

Getting AUDUSD.a's rates
Getting USDCAD.a's rates
Getting USDCHF.a's rates
Getting GBPUSD.a's rates
Getting USDJPY.a's rates
Getting NZDUSD.a's rates
Getting EURUSD.a's rates


In [5]:
combined_dfs['AU'].iloc[-1]

OBS_Value    103.38000
close          0.66199
Name: 2023-12-04 00:00:00, dtype: float64

In [6]:
combined_dfs['AU']

,OBS_Value,close
2014-05-06,119.94,0.93419
2014-05-07,120.05,0.93271
2014-05-08,120.62,0.93684
2014-05-09,120.85,0.93611
2014-05-12,121.07,0.93625
...,...,...
2023-11-28,102.84,0.66488
2023-11-29,102.66,0.66174
2023-11-30,102.67,0.66051
2023-12-01,103.02,0.66750


In [20]:
# Compute Expected Returns
weekly_dfs = {}
features = ['OBS_Value', 'close']

for name, dfs in combined_dfs.items():
    df = combined_dfs[name]
    # Assuming 'df' is your DataFrame
    df['date'] = pd.to_datetime(df.index)
    df.set_index('date', inplace=True)

    # Resample to get the last value of each week
    weekly = df.resample('W').last()
    weekly_dfs[name] = weekly

for df in weekly_dfs.values():
    df['EER_ret'] = df['OBS_Value'].pct_change()

In [23]:
for df in weekly_dfs.values():
    print(df['EER_ret'].iloc[-1])

0.0034944670937682165
0.00029667721518977785
0.004794885455513986
0.0019301293186642088
0.009685230024212954
0.0013718765311121484
-0.0030024019215372233


In [24]:
last_update = {}

for name, df in weekly_dfs.items():
    last_update[name] = df['EER_ret'].iloc[-1]

In [33]:
last_update.keys()
symbols = ['AUDUSD.a', 'USDCAD.a', 'USDCHF.a', 'GBPUSD.a', 'USDJPY.a', 'NZDUSD.a', 'EURUSD.a']

last_update = {symbols[i]: value for i, (key, value) in enumerate(last_update.items())}

In [34]:
last_update

{'AUDUSD.a': 0.0034944670937682165,
 'USDCAD.a': 0.00029667721518977785,
 'USDCHF.a': 0.004794885455513986,
 'GBPUSD.a': 0.0019301293186642088,
 'USDJPY.a': 0.009685230024212954,
 'NZDUSD.a': 0.0013718765311121484,
 'EURUSD.a': -0.0030024019215372233}

In [35]:
def send_order(symbol, side, lot, comment):
    
    if side.lower() == 'sell':
        order_type = mt5.ORDER_TYPE_SELL
        price = mt5.symbol_info_tick(symbol).bid
    elif side.lower() == 'buy':
        order_type = mt5.ORDER_TYPE_BUY
        price = mt5.symbol_info_tick(symbol).ask
    
    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": lot,
        "type": order_type,
        "price": price,
        "deviation": 5,
        "magic": 234000,
        "comment": comment,
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC,
    }
    result = mt5.order_send(request)
    result

In [37]:
for symbol, value in last_update.items():
    if value > 0:
        send_order(symbol, 'buy', 1.00, 'EER')
    else:
        send_order(symbol, 'sell', 1.00, 'EER')

In [38]:
# Work on automating it 